In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import joblib  
data = pd.read_csv("../data/project2.csv")
#display(data.head())


data["date"] = pd.to_datetime(data["date"], errors="coerce")
data["year"] = data["date"].dt.isocalendar().year
data["month"] = data["date"].dt.month
data["day"] = data["date"].dt.day
data["weekday"] = data["date"].dt.weekday
data["weekofyear"] = data["date"].dt.isocalendar().week

seasons = ["spring", "summer", "autumn", "winter"]
encoder = LabelEncoder()
encoder.fit(seasons)  # 全部を登録する

# データの season を変換
data["season"] = encoder.transform(data["season"])

X = data[["weekday", "month", "is_festival", "season",
          "weather_code", "temperature",
          "snowfall_sum", "rain_sum",
          "year", "day", "weekofyear"]]
y = data["sales"]

model = xgb.XGBRegressor(
    objective="reg:squarederror",
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100
)

display(X)


joblib.dump(encoder, "season_encoder0.1.pkl")

model.fit(X, y)
joblib.dump(model, "sales_model0_1.pkl")

,weekday,month,is_festival,season,weather_code,temperature,snowfall_sum,rain_sum,year,day,weekofyear
0,5,1,0,3,2.0,3.9,0.0,0.0,1999,1,52
1,6,1,0,3,3.0,5.0,0.0,0.0,1999,2,52
2,0,1,0,3,1.0,6.7,0.0,0.0,2000,3,1
3,1,1,0,3,0.0,5.2,0.0,0.0,2000,4,1
4,2,1,0,3,3.0,7.5,0.0,0.0,2000,5,1
...,...,...,...,...,...,...,...,...,...,...,...
9298,0,6,0,2,NaN,NaN,NaN,NaN,2025,16,25
9299,1,6,0,2,NaN,NaN,NaN,NaN,2025,17,25
9300,2,6,0,2,NaN,NaN,NaN,NaN,2025,18,25
9301,3,6,0,2,NaN,NaN,NaN,NaN,2025,19,25


['sales_model0_1.pkl']

['sales_model0_1.pkl']

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# -----------------------------
# データ読み込み & 前処理
# -----------------------------
data = pd.read_csv("/Users/khein21502/Documents/project_root/CCC_project/CCC_project/data/project2.csv")

# 保存済みモデルをロード
model = joblib.load("/Users/khein21502/Documents/project_root/CCC_project/CCC_project/model/sales_model0_1.pkl")

# 日付処理
data["date"] = pd.to_datetime(data["date"], errors="coerce")
data["year"] = data["date"].dt.isocalendar().year
data["month"] = data["date"].dt.month
data["day"] = data["date"].dt.day
data["weekday"] = data["date"].dt.weekday
data["weekofyear"] = data["date"].dt.isocalendar().week

# カテゴリ変数のエンコード（本来は学習時の encoder を joblib.load すべき）
encoder_season = LabelEncoder()
encoder_festival = LabelEncoder()
data["season"] = encoder_season.fit_transform(data["season"])
data["is_festival"] = encoder_festival.fit_transform(data["is_festival"])

# -----------------------------
# 特徴量と目的変数
# -----------------------------
X = data[[
    "weekday", "month", "is_festival", "season",
    "weather_code", "temperature",
    "snowfall_sum", "rain_sum",
    "year", "day", "weekofyear"
]]
y = data["sales"]

# -----------------------------
# データ分割
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# 予測 & 評価（学習済みモデルを利用）
# -----------------------------
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE :", mse)
print("RMSE:", rmse)
print("MAE :", mae)
print("R^2 :", r2)

# -----------------------------
# 特徴量重要度の可視化
# -----------------------------
xgb.plot_importance(model)
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/khein21502/Documents/project_root/CCC_project/CCC_project/model/sales_model.pkl'